In [1]:
import torch
import torchvision
import torchvision.datasets
import torchvision.transforms
import torch.utils.data
import torch.nn
import math
from torchvision import models as torch_model
import numpy as np
import torch.nn as nn
import graphviz
from torchview import draw_graph
import os
from matplotlib import pyplot as plot
import time

In [2]:
batch_size = 128
epochs = 5
learning_rate = 0.001
num_classes = 10
dir_name = os.getcwd()
acc_array = []

In [3]:
def get_norm_params(dataset):
    mean = dataset.data.mean(axis=(0,1,2))/255
    std = dataset.data.std(axis=(0,1,2))/255
    return torch.tensor(mean).reshape(3, 1, 1), torch.tensor(std).reshape(3, 1, 1)

In [4]:
def get_data_loader(transforms_params):

   train_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = True, download = True,
    transform = transforms_params
)
   test_dataset = torchvision.datasets.CIFAR10(
    root = dir_name, train = False, download = True,
    transform = transforms_params
)
   train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True
  )

   test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = batch_size, shuffle = False)
   return train_data_loader, test_data_loader

In [5]:
loss_function = torch.nn.CrossEntropyLoss()
print('Training parameters: learning rate = {}, batch size = {}, number of epochs = {}'.format(learning_rate, batch_size, epochs))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


Training parameters: learning rate = 0.001, batch size = 128, number of epochs = 5


device(type='cpu')

In [6]:
def get_accuracy(model, data_loader, device):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n

def get_loss(model, data_loader, device):
    loss = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            images = images.to(device)
            outputs = model(images)
            loss += loss_function(outputs, labels)
            n += labels.size(0)
    return loss / n

def accuracy(outputs, labels):
  _, predicted = torch.max(outputs.data, 1)
  return torch.tensor(torch.sum(predicted == labels).item() / len(predicted))

In [7]:
def train(Network, train_data_loader, test_data_loader, optimizer):
    for epoch in range(epochs):
        start = time.time()
        losses = []
        for i, (images, labels) in enumerate(train_data_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = Network(images)
            loss = loss_function(outputs, labels)
            losses.append(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        duration = time.time() - start
        print('Epoch[{}]: loss = {}, accuracy = {}, time = {}'.format(epoch, torch.stack(losses).mean(), get_accuracy(Network,train_data_loader, device), duration))
    acc = get_accuracy(Network,test_data_loader, device)
    print('Test accuracy: {}'.format(acc))
    return acc

In [8]:
def show_Network(Network):
  graphviz.set_jupyter_format('png')
  model_graph = draw_graph(Network, input_size=(1,3,32,32), expand_nested=True)
  model_graph.visual_graph

In [9]:
def FreezeLayers(Network, train_data_loader, test_data_loader, upd_layers):
  for params in Network.parameters():
    params.requires_grad = False
  Network.fc = torch.nn.Linear(Network.fc.in_features, upd_layers)
  return Network

In [10]:
def ModLastLAyers(Network, train_data_loader, test_data_loader, upd_layers):
    for params in Network.parameters():
     params.requires_grad = False
    Network.fc = torch.nn.Sequential(torch.nn.Linear(Network.fc.in_features, Network.fc.in_features//2), torch.nn.ReLU(), torch.nn.Linear(Network.fc.in_features//2, 10),)
    return Network

In [11]:
def get_first_ex(Network_type, weights):
  Network = Network_type(weights)
  transforms = weights.transforms()
  train_data_loader, test_data_loader = get_data_loader(transforms)
  Network.to(device)
  Network = FreezeLayers(Network, train_data_loader, test_data_loader, num_classes)
  Network.to(device)
  optimizer = torch.optim.Adam(Network.parameters(), lr = learning_rate)
  acc = train(Network, train_data_loader, test_data_loader, optimizer)
  print(Network)
  show_Network(Network)
  return acc

In [12]:
def get_second_ex(Network_type, weights):
  Network = Network_type(weights)
  transforms = weights.transforms()
  train_data_loader, test_data_loader = get_data_loader(transforms)
  Network = FreezeLayers(Network, train_data_loader, test_data_loader, num_classes)
  Network.to(device)
  optimizer = torch.optim.Adam(Network.parameters(), lr = learning_rate)
  acc = train(Network, train_data_loader, test_data_loader, optimizer)
  print(Network)
  show_Network(Network)
  return acc

In [13]:
def show_exp_result(acc_array_, type_array_):
  results = {}
  for i in range(len(acc_array_)):
    results[type_array[i]] = acc_array[i].cpu()
  print(results)
  fig = plot.figure(figsize = (25, 6))
  plot.bar(results.keys(), results.values())
  plot.xlabel("Models experiments")
  plot.ylabel("Test accuracy")
  plot.title("Comparison of architectures modifications")
  for i in range(len(acc_array_)):
        plot.text(i, acc_array[i].cpu() + 0.01, f'{int(acc_array[i].cpu() * 100)} %', ha='center')
  plot.show()

In [14]:
acc_array.append(get_first_ex(torch_model.googlenet, torch_model.GoogLeNet_Weights.DEFAULT))

C:\Users\soup4\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 1.0408164262771606, accuracy = 0.7414199709892273, time = 1835.5575098991394
Epoch[1]: loss = 0.7561564445495605, accuracy = 0.7610399723052979, time = 2448.220628261566
Epoch[2]: loss = 0.714107096195221, accuracy = 0.7603800296783447, time = 1957.750419139862
Epoch[3]: loss = 0.6976123452186584, accuracy = 0.766979992389679, time = 2046.6039657592773
Epoch[4]: loss = 0.6846154928207397, accuracy = 0.773419976234436, time = 1815.260261774063
Test accuracy: 0.7544999718666077
GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(

In [15]:
acc_array.append(get_second_ex(torch_model.googlenet, torch_model.GoogLeNet_Weights.DEFAULT))

Files already downloaded and verified
Files already downloaded and verified
Epoch[0]: loss = 1.0491406917572021, accuracy = 0.7389799952507019, time = 1802.4770665168762
Epoch[1]: loss = 0.7557076215744019, accuracy = 0.7572199702262878, time = 1797.7528262138367
Epoch[2]: loss = 0.7098788022994995, accuracy = 0.7639399766921997, time = 1801.5689315795898
Epoch[3]: loss = 0.6939578652381897, accuracy = 0.7678200006484985, time = 1800.108996629715
Epoch[4]: loss = 0.6847813725471497, accuracy = 0.770799994468689, time = 1803.5040018558502
Test accuracy: 0.7541999816894531
GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNor

In [ ]:
acc_array.append(get_first_ex(torch_model.resnet50, torch_model.ResNet50_Weights.DEFAULT))

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\soup4/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:09<00:00, 11.3MB/s]


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
acc_array.append(get_second_ex(torch_model.resnet50, torch_model.ResNet50_Weights.DEFAULT))

In [ ]:
acc_array.append(get_first_ex(torch_model.resnet34, torch_model.ResNet34_Weights.DEFAULT))

In [ ]:
acc_array.append(get_second_ex(torch_model.resnet34, torch_model.ResNet34_Weights.DEFAULT))

In [ ]:
acc_array.append(get_first_ex(torch_model.shufflenet_v2_x2_0, torch_model.ShuffleNet_V2_X2_0_Weights.DEFAULT))

In [ ]:
acc_array.append(get_second_ex(torch_model.shufflenet_v2_x2_0, torch_model.ShuffleNet_V2_X2_0_Weights.DEFAULT))

In [ ]:
acc_array

In [ ]:
type_array = ("GoogleNet First", "GoogleNet Second", "ResNet50 First", "ResNet50 Second", "ResNet34 First", "ResNet34 Second", "Shufflenet_v2_x2_0 First", "Shufflenet_v2_x2_0 Second")


In [ ]:
show_exp_result(acc_array, type_array)